In [53]:
import cv2 as cv

image = cv.imread("../data/images/road818.png")

In [4]:
def show_img(img, title:str="Image"):
    cv.imshow(title, img)
    cv.waitKey(0)
    cv.destroyWindow(title)

In [8]:
def contrast(bgr_image):
    ycrcb_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2YCrCb)
    
    y, cr, cb = cv.split(ycrcb_image)
    y_equalized = cv.equalizeHist(y)
    equalized_image = cv.merge([y_equalized, cr, cb])
    return cv.cvtColor(equalized_image, cv.COLOR_YCrCb2BGR)

In [24]:
def automatic_brightness_contrast(bgr_image, clip_hist_percent = 0.01, use_scale_abs = True, return_verbose = False):
    gray_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2GRAY)

    # Grayscale histogram of the image
    hist = cv.calcHist([gray_image], [0], None, [256], [0, 256])
    hist_size = len(hist)

    # Cumulative distribution of the histogram
    acc = []
    acc.append( float(hist[0]) )
    for i in range(1, hist_size):
        acc.append( acc[i - 1] + float(hist[i]) )
    
    # Locate points to clip
    maximum = acc[-1]
    clip_hist = clip_hist_percent * maximum / 2.0

    # Left cut
    minimum_gray = 0
    while acc[minimum_gray] < clip_hist:
        minimum_gray += 1

    # Right cut
    maximum_gray = hist_size - 1
    while acc[maximum_gray] >= (maximum - clip_hist):
        maximum_gray -= 1

    # Calculate alpha and beta values for the scaling
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = - minimum_gray * alpha

    if use_scale_abs:
        processed_image = cv.convertScaleAbs(bgr_image, alpha=alpha, beta=beta)
    else:
        processed_image = bgr_image * alpha + beta
        processed_image[processed_image < 0] = 0
        processed_image[processed_image > 255] = 255

    if return_verbose:
        processed_hist = cv.calcHist([gray_image], [0], None, [256], [minimum_gray, maximum_gray])

        return processed_image, alpha, beta, hist, processed_hist
    
    return processed_image

In [34]:
def LaplacianOfGaussian(bgr_image):
    log_image = cv.GaussianBlur(bgr_image, (3, 3), 0)
    gray_image = cv.cvtColor(log_image, cv.COLOR_BGR2GRAY)
    log_image = cv.Laplacian(log_image, cv.CV_8U, ksize=3, scale=1, delta=0)
    return cv.convertScaleAbs(log_image)

In [62]:
show_img(image)

new_img = automatic_brightness_contrast(image, clip_hist_percent=0.1, use_scale_abs=True)

#new_img = LaplacianOfGaussian(new_img)
show_img(new_img)

In [46]:
def extract_red_hsv(hsv_image, return_split = False):
    # First zone of reds
    lowerbound_1 = np.array([0, 40, 25])
    upperbound_1 = np.array([10, 255, 255])

    # Second zone of reds
    lowerbound_2 = np.array([135, 40, 25])
    upperbound_2 = np.array([179, 255, 255])

    red_1 = cv.inRange(hsv_image, lowerbound_1, upperbound_1)
    red_2 = cv.inRange(hsv_image, lowerbound_2, upperbound_2)

    if return_split:
        return red_1, red_2
    
    return cv.bitwise_or(red_1, red_2)

def extract_blue_hsv(hsv_image):
    lowerbound = np.array([100, 160, 40])
    upperbound = np.array([120, 255, 255])

    return cv.inRange(hsv_image, lowerbound, upperbound)

In [47]:
def binarize(bgr_image, threshold_percent = 0.75, return_thresh = False):
    gray_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2GRAY)

    threshold = int(threshold_percent * 255)

    thresh, thresh_image = cv.threshold(gray_image, threshold, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    if return_thresh:
        return thresh_image, thresh
    return thresh_image

In [105]:
def extractObjects(bgr_image, contours):
    out_image = np.zeros_like(bgr_image)

    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        out_image[y:y+h, x:x+w] = bgr_image[y:y+h, x:x+w]
    
    return out_image

In [104]:
show_img(red_img)

In [106]:
objects = extractObjects(image, contours)

In [119]:
contrast_image = automatic_brightness_contrast(objects, 0.05)

In [129]:
red_mask = extract_red_hsv(objects)

white_mask = cv.inRange(cv.cvtColor(objects, cv.COLOR_BGR2HSV), np.array([0, 0, 130]), np.array([255, 40, 255]))

mask = cv.bitwise_or(red_mask, white_mask)

cropped_objects = cv.bitwise_and(objects, objects, mask=mask)
show_img(objects)
show_img(cropped_objects)

bin_image = binarize(cropped_objects)

In [127]:
show_img(bin_image)
kernel = np.ones((3, 3), np.uint8)
morph_img = cv.morphologyEx(bin_image, cv.MORPH_DILATE, kernel)
show_img(morph_img)

In [ ]:
# WARMING FILTER
# https://towardsdatascience.com/python-opencv-building-instagram-like-image-filters-5c482c1c5079

def _create_LUT_8UC1(x, y):
  spl = UnivariateSpline(x, y)
  return spl(range(256))

# TODO better lut values for reds
incr_ch_lut = _create_LUT_8UC1([0, 64, 128, 256],[0, 80, 160, 256])
decr_ch_lut = _create_LUT_8UC1([0, 64, 128, 256],[0, 50,  100, 255])

def render(img_rgb):
    c_r, c_g, c_b = cv.split(img_rgb)
    c_r = cv.LUT(c_r, incr_ch_lut).astype(np.uint8)
    c_b = cv.LUT(c_b, decr_ch_lut).astype(np.uint8)
  
    return  cv.merge((c_r, c_g, c_b)) 

bla = cv.cvtColor(src_image, cv.COLOR_BGR2RGB)  
display_bgr_image(render(bla))

src_image = bla

In [ ]:
def get_illumination_channel(I, w):
    M, N, _ = I.shape
    w_2 = int(w/2)
    padded = np.pad(I, ( (w_2, w_2), (w_2, w_2), (0, 0)), "edge")
    dark_channel = np.zeros(shape=(M, N))
    bright_channel = np.zeros(shape=(M, N))

    for i, j in np.ndindex(dark_channel.shape):
        dark_channel[i, j] = np.min(padded[i:i+w, j:j+w, :])
        bright_channel[i, j] = np.max(padded[i:i+w, j:j+w, :])

    return dark_channel, bright_channel

def get_atmosphere(I, bright_channel, p=0.1):
    M, N = bright_channel.shape
    flatI = I.reshape(M*N, 3)
    flatBright = bright_channel.ravel()

    search_idx = (-flatBright).argsort()[:int(M*N*p)]
    return np.mean(flatI.take(search_idx, axis=0), dtype=np.float64, axis=0)

def get_initial_transmission(A, bright_channel):
    A_c = np.max(A)
    init_t = (bright_channel - A_c) / (1.0 - A_c)
    return (init_t - np.min(init_t)) / (np.max(init_t) - np.min(init_t))

def reduce_init_t(init_t):
    init_t = np.uint8(init_t * 255)
    xp = [0, 32, 255]
    fp = [0, 32, 48]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype(np.uint8)
    init_t = cv.LUT(init_t, table)
    return np.float64(init_t) / 255

def corrected_transmission(I, A, dark_channel, bright_channel, init_t, alpha, omega, w):
    im = np.empty(I.shape, I.dtype)
    for ind in range(0, 3):
        im[:, :, ind] = I[:, :, ind] / A[ind]
    
    dark_c, _ = get_illumination_channel(im, w)
    dark_t = 1 - omega * dark_c
    corrected_t = init_t
    diff_channel = bright_channel - dark_channel
    for i in range(diff_channel.shape[0]):
        for j in range(diff_channel.shape[1]):
            if diff_channel[i, j] < alpha:
                corrected_t[i, j] = dark_t[i, j] * init_t[i, j]

    return np.abs(corrected_t)

def boxfilter(I, r):
    """Fast box filter implementation.
    Parameters
    ----------
    I:  a single channel/gray image data normalized to [0.0, 1.0]
    r:  window radius
    Return
    -----------
    The filtered image data.
    """
    M, N = I.shape
    dest = np.zeros((M, N))
    
    sumY = np.cumsum(I, axis=0)
    # difference over Y axis
    dest[:r + 1] = sumY[r:2*r + 1] # top r+1 lines
    dest[r + 1:M - r] = sumY[2*r + 1:] - sumY[:M - 2*r - 1]
    dest[-r:] = np.tile(sumY[-1], (r, 1)) - sumY[M - 2*r - 1:M - r - 1] # bottom r lines

    sumX = np.cumsum(dest, axis=1)
    # difference over X axis
    dest[:, :r + 1] = sumX[:, r:2*r + 1] # left r+1 columns
    dest[:, r + 1:N - r] = sumX[:, 2*r + 1:] - sumX[:, :N - 2*r - 1]
    dest[:, -r:] = np.tile(sumX[:, -1][:, None], (1, r)) - sumX[:, N - 2*r - 1:N - r - 1] # right r columns

    return dest


def guided_filter(I, p, r=15, eps=1e-3):
    """Refine a filter under the guidance of another (RGB) image.
    Parameters
    -----------
    I:   an M * N * 3 RGB image for guidance.
    p:   the M * N filter to be guided. transmission is used for this case.
    r:   the radius of the guidance
    eps: epsilon for the guided filter
    Return
    -----------
    The guided filter.
    """
    from itertools import combinations_with_replacement
    R, G, B = 0, 1, 2

    M, N = p.shape
    base = boxfilter(np.ones((M, N)), r) # this is needed for regularization
    
    # each channel of I filtered with the mean filter. this is myu.
    means = [boxfilter(I[:, :, i], r) / base for i in range(3)]
    
    # p filtered with the mean filter
    mean_p = boxfilter(p, r) / base

    # filter I with p then filter it with the mean filter
    means_IP = [boxfilter(I[:, :, i]*p, r) / base for i in range(3)]

    # covariance of (I, p) in each local patch
    covIP = [means_IP[i] - means[i]*mean_p for i in range(3)]

    # variance of I in each local patch: the matrix Sigma in ECCV10 eq.14
    var = defaultdict(dict)
    for i, j in combinations_with_replacement(range(3), 2):
        var[i][j] = boxfilter(I[:, :, i]*I[:, :, j], r) / base - means[i]*means[j]

    a = np.zeros((M, N, 3))
    for y, x in np.ndindex(M, N):
        #         rr, rg, rb
        # Sigma = rg, gg, gb
        #         rb, gb, bb
        Sigma = np.array([[var[R][R][y, x], var[R][G][y, x], var[R][B][y, x]],
                          [var[R][G][y, x], var[G][G][y, x], var[G][B][y, x]],
                          [var[R][B][y, x], var[G][B][y, x], var[B][B][y, x]]])
        cov = np.array([c[y, x] for c in covIP])
        a[y, x] = np.dot(cov, np.linalg.inv(Sigma + eps*np.eye(3)))  # eq 14

    # ECCV10 eq.15
    b = mean_p - a[:, :, R]*means[R] - a[:, :, G]*means[G] - a[:, :, B]*means[B]

    # ECCV10 eq.16
    q = (boxfilter(a[:, :, R], r)*I[:, :, R] + boxfilter(a[:, :, G], r)*I[:, :, G] + boxfilter(a[:, :, B], r)*I[:, :, B] + boxfilter(b, r)) / base

    return q

def get_final_image(I, A, refined_t, Tmin):
    refined_t_broadcasted = np.broadcast_to(refined_t[:, :, None], (refined_t.shape[0], refined_t.shape[1], 3))
    J = (I - A) / (np.where(refined_t_broadcasted < Tmin, Tmin, refined_t_broadcasted)) + A
    return (J - np.min(J)) / (np.max(J) - np.min(J))

def dehaze(image, Tmin = 0.1, w = 15, alpha = 0.4, omega = 0.75, p=0.1, eps=1e-3):
    I = np.float64(image)
    I = I[:, :, :3] / 255
    M, N, _ = I.shape
    Idark, Ibright = get_illumination_channel(I, w)
    A = get_atmosphere(I, Ibright, p)

    init_t = get_initial_transmission(A, Ibright)

    init_t = reduce_init_t(init_t)

    corrected_t = corrected_transmission(I, A, Idark, Ibright, init_t, alpha, omega, w)
    normI = (I - I.min()) / (I.max() - I.min())

    refined_t = guided_filter(normI, corrected_t, w, eps)
    J_refined = get_final_image(I, A, refined_t, Tmin)

    enhanced = np.uint8(J_refined * 255)
    f_enhanced = cv.detailEnhance(enhanced, sigma_s=10, sigma_r=0.15)
    f_enhanced = cv.edgePreservingFilter(f_enhanced, flags=cv.RECURS_FILTER, sigma_s=64, sigma_r=0.2)
    return f_enhanced